In [1]:
import math

import numpy as np
from scipy.stats import f
from scipy.stats import norm, gamma, laplace

## 1.3
$X_1 ... X_n \sim N(\mu_1, \sigma_1^2)$\
$Y_1 ... Y_m \sim N(\mu_2, \sigma_2^2)$\
$X, Y - $ независимы

$\tau = \frac{\sigma_1^2}{\sigma_2^2}, \mu_1, \mu_2$ - незвестны

По теореме Фишера:

$U_1 = \frac{n S_{1*}^2}{\sigma_1^2} \sim \chi (n-1)$\
$U_2 = \frac{n S_{2*}^2}{\sigma_2^2} \sim \chi (m-1)$

Воспользуемся распределением Фишера

$\frac{\frac{U_2}{m-1}}{\frac{U_1}{n-1}} \sim F(n, m)$\
$\frac{m(n-1)S_{2*}^2 \sigma_1^2}{n(m-1)S_{1*}^2 \sigma_2^2} \sim F(n, m)$\
$P(q_{\frac{\alpha}{2}} <= \frac{m(n-1)S_{2*}^2 \sigma_1^2}{n(m-1)S_{1*}^2 \sigma_2^2} <= q_{1 - \frac{\alpha}{2}})$
$P(\frac{\sigma_1^2}{\sigma_2^2} in [q_{\frac{\alpha}{2}} \frac{n(m-1)S_{1*}^2}{m(n-1)S_{2*}^2}; q_{1 - \frac{\alpha}{2}}\frac{n(m-1)S_{1*}^2}{m(n-1)S_{2*}^2}])$


In [17]:
mu1 = 0
mu2 = 0
sigma1=2
sigma2=1
n = 25
m = 25
alpha = 0.05
n_s = [25, 10000]
for it in n_s:
    cnt = 0
    n = m = it
    F_lower = f.ppf(alpha/2, n-1, m-1)
    F_upper = f.ppf(1 - alpha/2, n-1, m-1)
    for i in range(1000):
        X = norm.rvs(loc=mu2, scale=math.sqrt(sigma1),size=n)
        Y = norm.rvs(loc=mu2, scale=sigma2, size=m)
        var_X = np.var(X)
        var_Y = np.var(Y)
        q = n*(m-1) * var_X / (m*(n-1)*var_Y)
        if 2 <= F_upper * q and 2 >= F_lower *q:
            cnt += 1
    print("Stats: ", cnt, "from ", 1000)


Stats:  945 from  1000
Stats:  951 from  1000


Результаты эксперемента показывают, что доверительный интервал найден верно. Попаданий больше, чем 95%.

Процент попадания осталься примерно таким же.

## 2.2

Пусть $X_1, \dots, X_n \sim \mathrm{Laplace}(\mu, \alpha)$,  
где $\mu$ — параметр сдвига (неизвестен), а масштаб $\alpha$ известен.



### Из теоремы о средних членах вариационного ряда:
$$
n F(X_{(n/2)}; \mu) \xrightarrow{d} \Gamma(n/2, 1),
$$

где $X_{(n/2)}$ — медиана,  
а $F(x;\mu)$ — функция распределения.

Тогда, выразив $\mu$ из двойного неравенства, найдём асимптотический доверительный интервал:
$$
P\!\big(q_{\alpha/2} \le n F(x_r; \mu) \le q_{1-\alpha/2}\big) = 1 - \alpha,
$$
где $q_p$ — $p$-й квантиль распределения $\Gamma(n/2,1)$.

функция распределения Лапласа имеет вид:
$$
F(x;\mu) =
\begin{cases}
\dfrac{1}{2} e^{\alpha (x - \mu)}, & x < \mu, \\
1 - \dfrac{1}{2} e^{-\alpha (x - \mu)}, & x \ge \mu.
\end{cases}
$$



In [21]:
mu = 2
scale = 1
alpha = 0.05
n = 10000
n_s = [25, 10000]
for it in n_s:
    cnt = 0
    n = m = it
    left_quantile = gamma.ppf(alpha/2, n/2, 1)
    right_quantile = gamma.ppf(1 - alpha/2, n/2, 1)
    for i in range(1000):
        X = laplace.rvs(loc=mu, scale=scale, size=n)
        med_X = np.median(X)
        mu_L1 = med_X - scale * np.log(2 * right_quantile / n)
        mu_U1 = med_X - scale * np.log(2 * left_quantile / n)
        mu_L2 = med_X + scale * np.log(2 * (1 - right_quantile / n))
        mu_U2 = med_X + scale * np.log(2 * (1 - left_quantile / n))
        mu_L = min(mu_L1, mu_L2)
        mu_U = max(mu_U1, mu_U2)
        if mu_L <= mu <= mu_U:
            cnt += 1
    print("Stats: ", cnt, "from ", 1000)


Stats:  982 from  1000
Stats:  991 from  1000
